## Jackknife Bias Estimation for Morphometric Connectivity

This notebook describes the steps we used for calculating structural covariance and graph measures from CIVET-based cortical thickness:

1. Install & Import Packages
2. Read CIVET Files
3. DKT Parcellation
4. Covariate Regression
5. Structural Covariance
6. Jackknife Bias Estimation
7. Graph Theory

MATLAB-Code available [here](https://github.com/katielavigne/civetsurf).

### 1. Data Preparation

#### 1a. Install & Import Packages 

This step ensures all needed packages are either installed or, if already installed, imported.

In [ ]:
# install packages needed for all steps of the analyses
!pip install bctpy
!pip install statsmodels
!pip install pandas 
!pip install numpy

In [ ]:
# import packages
import os, glob
import pandas as pd
import numpy as np 

In [ ]:
import bct
import statsmodels.formula.api as sm

#### 1b. Define Paths & Files

Modify this step for your specific data.

In [ ]:
# define paths, files & variables
civpath = '/scratch/katie/ukbb/data/civet_0mm/' # location containing civet outputs
dpath = '/scratch/katie/ukbb/data/' # location to load glimfile & save outputs

gfile = 'ukbb_glimfile_final.csv' # glimfile
dktvert_file = 'CIVET_2.0_DKT.txt' # DKT CIVET-based vertex file
dktinfo_file = 'DKT.csv' # DKT parcellation info

measure = 'thickness' # structural metric (e.g., thickness, surface_area, volume)
idvar = 'eid' # unique subject ID variable
group = 'dx' # grouping variable (if applicable)
covars = ['age','mean_' + measure] # covariates

### 2. Read Files

This step reads [CIVET](https://www.bic.mni.mcgill.ca/ServicesSoftware/CIVET-2-1-0-Table-of-Contents)-based cortical surface text files from a specified directory. CIVET outputs text files consisting of  values for a given structural metric (cortical thickness, surface area, volume) for each vertex separately for each hemisphere. Using cortical thickness as an example with data from the [UK BioBank](https://www.ukbiobank.ac.uk/), this script loads the files for both hemispheres and merges them into a dataframe displaying all vertices. Further, mean cortical metric values are calculated for each participant by taking the mean of all vertices per participant, and a total metric value is calculated by summing the values of each participant. Depending on the cortical metric, one would use either the mean (thickness) or total (surface area) values for later steps.

Inputs are:
- cortical thickness files for each hemisphere of N participants

Outputs are:
- a dataframe with the dimensions N x 81,924 vertices
- a dataframe with mean thickness values
- a dataframe with total thickness values 

___

For high performance computing, see:
- readfiles.py
- readfiles.sh

In [ ]:
# find files & IDs
Lfiles = glob.glob(civpath + '*left*')
Lfiles.sort()
Rfiles = glob.glob(civpath + '*right*')
Rfiles.sort()
subjIDs = [ids.split('/')[-1].split('_')[1] for ids in Lfiles] # modify this for different filename conventions

# make dataframe & save as pickle
Ldf = pd.concat((pd.read_csv(Lf, dtype=float, header=None).T for Lf in Lfiles))
Rdf = pd.concat((pd.read_csv(Rf, dtype=float, header=None).T for Rf in Rfiles))
df = pd.concat([Ldf,Rdf], axis=1)
df.index = subjIDs
df.index.names = [idvar]
df.to_pickle(dpath + measure + '_vertexdata.pkl')

# calculate and save mean anatomical measure (mean thickness)
mean_measure = pd.DataFrame(pd.to_numeric(df.mean(axis=1)), columns=['mean_' + measure])
mean_measure.to_csv(dpath + 'mean_' + measure + '.csv')

# calculate and save total anatomical measure (total thickness)
tot_measure = pd.DataFrame(pd.to_numeric(df.sum(axis=1)), columns=['total_' + measure])
tot_measure.to_csv(dpath + 'total_' + measure + '.csv')

### 3. DKT-Parcellation 

This step parcellates the data of the 81,924 vertices into 62 DKT-regions ([Klein & Tourville, 2012](https://doi.org/10.3389/fnins.2012.00171)).

Inputs are:
- the previously created dataframe with all 81,924 vertices 
- a DKT-file indicating the DKT regions (see the separate file "DKT.csv" in the repository; from [MATLAB code](https://github.com/katielavigne/civetsurf) and the [CIVET Manual](https://www.bic.mni.mcgill.ca/ServicesSoftware/CIVET-2-1-0-Table-of-Contents))
- a DKT-file indicating which vertex belongs to which DKT-region (see the separate file "CIVET_2.0_DKT.txt" in the repository; from [MATLAB code](https://github.com/katielavigne/civetsurf) and the [CIVET Manual](https://www.bic.mni.mcgill.ca/ServicesSoftware/CIVET-2-1-0-Table-of-Contents))

Outputs are:
- a dataframe with the parcellated data of the dimensions N x 62 

___

For high performance computing see:
- parcellation.py
- parcellation.sh

In [ ]:
# read DKT and data files
#df = pd.read_pickle(dpath + measure + '_vertexdata.pkl')
dktvert = pd.read_csv(dktvert_file, dtype=str, names=['roi'])
dktvert

dktinfo = pd.read_csv(dktinfo_file, dtype=str)

# parcellation
parc = pd.DataFrame(index= df.index.copy())
for r in range(len(dktinfo)):
    roi = dktinfo.label_number[r]
    abr = dktinfo.abbreviation[r]
    means = pd.DataFrame(df.iloc[:,dktvert.index[dktvert.roi == roi]].mean(axis=1),columns=[abr], index= df.index.copy())
    parc = pd.concat([parc,means], axis = 1)

# write to csv
parc.to_csv(dpath + 'dkt_parcellation_' + measure + '.csv') # parcellated data

### 4. Regression

This (optional) step regresses out the influence of other variables on the cortical thickness values, resulting in residual values that can be used in following steps.

Inputs are:
- the parcellated data (N x 62)
- a glimfile including other variables (e.g., age for this example)
- the mean thickness values computed above

Outpus are:
- a dataframe with the residuals of the dimensions N x 62

___

For high performance computing see:
- regression.py
- regression.sh

In [ ]:
# read files
glim = pd.read_csv(dpath + gfile, index_col='eid', dtype={'eid':str})
parc = pd.read_csv(dpath + 'dkt_parcellation_' + measure + '.csv', index_col='eid', dtype={'eid':str})
mean_measure = pd.read_csv(dpath + 'mean_' + measure + '.csv', index_col='eid', dtype={'eid':str})
glim = glim[glim.index.isin(parc.index)]

# merge glimfile and parcellated data
glim = glim.join(mean_measure) # merge glim & mean thickness # modify to total_measure if using surface_area
glim_parc = glim.join(parc) # join glim + mean thickness & parcellated data
rois = parc.columns

# regression
if group in glim_parc.columns: # group-based regressions if applicable
    for g in glim_parc[group].unique():
        resid=[]
        glim_parc_group = glim_parc[glim_parc[group]==g]
        parc_group = glim_parc_group[rois]
        covar1 = glim_parc_group[covars[0]]
        covar2 = glim_parc_group[covars[1]]
        for i in parc_group:
            reg = sm.ols('parc_group.loc[:,i] ~ covar1 + covar2', data=glim_parc_group).fit()
            residuals = reg.resid
            resid.append(residuals)
        
        # merge groups
        if g == glim_parc[group].unique()[0]:
            res = pd.DataFrame(resid).T
            res.columns = parc.columns
        else:
            tmp = pd.DataFrame(resid).T
            tmp.columns = parc.columns
            res = pd.concat([res,tmp], sort=True)
else:
    resid=[]
    covar1 = glim_parc[covars[0]]
    covar2 = glim_parc[covars[1]]
    for i in parc:
        reg = sm.ols('parc.loc[:,i] ~ covar1 + covar2', data=glim_parc).fit()
        residuals = reg.resid
        resid.append(residuals)
    res = pd.DataFrame(resid).T
    res.columns = parc.columns

# write to csv
glim.to_csv(dpath + 'glimfile.csv')
glim_parc.to_csv(dpath + 'glimfile_parcellation.csv')
res.to_csv(dpath + 'residuals.csv')

### 5. Structural Covariance 

This step performs pearson correlations between DKT regions for all participants, resulting in a structural covariance matrix of the full sample ([Alexander-Bloch et al., 2013a](https://doi.org/10.1038/nrn3465); [Alexander-Bloch et al., 2013b](https://doi.org/10.1523/JNEUROSCI.3554-12.2013); [Evans, 2013](https://doi.org/10.1016/j.neuroimage.2013.05.054)).

Inputs are:
- the residual data of the DKT regions

Outputs are:
- a structural covariance matrix of the full sample with the dimensions 62 x 62

___

For high performance computing see:
- strucov.py
- strucov.sh

In [ ]:
# read files
#glim_parc = pd.read_csv(dpath + 'glimfile_parcellation.csv', index_col=[idvar], dtype={'eid':str})
#res = pd.read_csv(dpath + 'residuals.csv', index_col=[idvar], dtype={'eid':str}) # read residuals

# full sample (or group) correlation matrix
if group in glim_parc.columns:
    for g in glim_parc[group].unique():
        res_group = res.loc[glim_parc[group]==g]
        corrmtrix = res_group.corr(method='pearson')
        corrmtrix.to_csv(dpath + 'corrmtrix_full_group_' + str(g) + '.csv') # write to csv
else:
    corrmtrix = res.corr(method='pearson')
    corrmtrix.to_csv(dpath + 'corrmtrix_full.csv') # write to csv

### 6. Jackknife Bias Estimation Procedure 

This step calculates the contribution of each participant to the structural covariance matrix of the full sample ([Ajnakina et al., 2021](https://doi.org/10.1093/schbul/sbab035); [Das et al., 2018](https://doi.org/10.1001/jamapsychiatry.2018.0391)). Here, a structural covariance matrix is recalculated for each iteration of N-1 participants, meaning that the structural covariance between DKT regions is recalculated N times, by leaving each participant out of the calculation once. By then subtracting these recalculated matrices from the structural covariance matrix of the full sample, the contribution of each participant to the full-sample structural covariance is estimated. Absolute values are taken and the end result is a 3D matrix with the dimensions 62 x 62 x N.

Inputs are:
- the residual data of the DKT regions
- the structural covariance matrix of the full sample

Outputs are:
- an absolute value 3D matrix of the dimensions 62 x 62 x N
    - each of the N matrices represents the absolute contribution of each participant to the full-sample structural covariance matrix

*NOTE.* For analyses with multiple groups (e.g., patient & control), jackknife should be performed on each group separately.
___

For high performance computing see:
- jackknife.py
- jackknife.sh

In [ ]:
# read files
#glim_parc = pd.read_csv(dpath + 'glimfile_parcellation.csv', index_col=[idvar], dtype={'eid':str})
#res = pd.read_csv(dpath + 'residuals.csv', index_col=[idvar], dtype={'eid':str})

# jackknife correlation 
if group in glim_parc.columns:
    for g in glim_parc[group].unique():
        glim_parc_group = glim_parc[glim_parc[group]==g]
        res_group = res.loc[glim_parc[group]==g]
        corrmtrix = pd.read_csv(dpath + 'corrmtrix_full_group_' + str(g) + '.csv', index_col=[0])
        jack = []
        n = len(glim_parc_group[:]) # full sample
        for i,row in res_group.iterrows(): # i = eid, so the loop goes over each row in the dataframe meaning each participant
            LOO = res_group.drop(i) # and drops one row depending on i per loop (LOO = leave one out)
            corrLOO = LOO.corr(method='pearson');
            W = (n*corrmtrix)-((n-1)*corrLOO);
            # Absolute
            normW = abs(W);
            jack.append(normW)
        jk = np.array(jack)
        np.save(dpath + 'jackknife_output_group_' + str(g) + '.npy', jk) # write out as numpy array
else:
    jack = []
    n = len(glim_parc[:]) # full sample
    corrmtrix = pd.read_csv(dpath + 'corrmtrix_full.csv', index_col=[0])
    for i,row in res.iterrows(): # i = eid, so the loop goes over each row in the dataframe meaning each participant
        LOO = res.drop(i) # and drops one row depending on i per loop (LOO = leave one out)
        corrLOO = LOO.corr(method='pearson'); 
        W = (n*corrmtrix)-((n-1)*corrLOO);
        # Absolute
        normW = abs(W);
        jack.append(normW)
    jk = np.array(jack)
    np.save(dpath + 'jackknife_output.npy', jk) # write out as numpy array

### 7. Graph Theory 

This step calculates the two graph measures of local strength and global efficiency by the means of the [Brain Connectivity Toolbox](http://www.brain-connectivity-toolbox.net/) in Python ([bctpy](https://pypi.org/project/bctpy/); [Rubinov & Sporns, 2010](https://doi.org/10.1016/j.neuroimage.2009.10.003)).

Inputs are:
- the 3D jackknife connectivity matrix 
- the glimfile

Outputs are:
- local strengths
- global efficiency 

___

For high performance computing see:
- graph.py
- graph.sh

In [ ]:
# read files
glim = pd.read_csv(dpath + 'glimfile.csv', index_col=[idvar], dtype={'eid':str})

# compute graph measures
if group in glim.columns:
    for g in glim[group].unique():
        glim_group = glim[glim[group]==g]
        jk = np.load(dpath +'jackknife_output_group_' + str(g) + '.npy')
        
        # fixing that the diagonals are all 0 
        for i in jk:
            np.fill_diagonal(i, 0, wrap=True)
        
        # strengths
        def strengths_und(jk):
            return np.sum(jk, axis=1)
        strengths = strengths_und(jk)
        s = pd.DataFrame(data=strengths, index=glim_group.index.copy())
        s.columns = ['Strength_'+ str(i) for i in range(1, s.shape[1] + 1)]
        tmp = glim_group.join(s)
        
        # global efficiency
        globeff = []
        for i in jk:
            bct.efficiency_wei(i)
            globeff.append(bct.efficiency_wei(i))
        e = pd.DataFrame(data=globeff, index=glim_group.index.copy())
        e.columns = ['Global Efficiency']
        tmp = tmp.join(e)
    
        # merge groups
        if g == glim[group].unique()[0]:
            data_conn = tmp
        else:
            data_conn = pd.concat([data_conn,tmp])
    
    # write to csv
    data_conn.to_csv(dpath + 'glimfile_jackknife_output.csv')
else:
    jk = np.load(dpath + 'jackknife_output.npy')
    
    # fixing that the diagonals are all 0
    for i in jk:
        np.fill_diagonal(i, 0, wrap=True)
    
    # strengths
    def strengths_und(jk):
        return np.sum(jk, axis=1)
    strengths = strengths_und(jk)
    s = pd.DataFrame(data=strengths, index=glim.index.copy())
    s.columns = ['Strength_'+str(i) for i in range(1, s.shape[1] + 1)]
    data_conn = glim.join(s)
    
    # global efficiency
    globeff = []
    for i in jk:
        bct.efficiency_wei(i)
        globeff.append(bct.efficiency_wei(i))
    e = pd.DataFrame(data=globeff, index=glim.index.copy())
    e.columns = ['Global Efficiency']
    data_conn = data_conn.join(e)
    
    # write to csv
    data_conn.to_csv(dpath + 'glimfile_jackknife_output.csv')